In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import requests
from PIL import Image
from io import BytesIO

In [ ]:
#plot the players picture
# Define the base URL for player photos
photo_base_url = "https://resources.premierleague.com/premierleague/photos/players/110x140/p"

goalies_df = pd.read_csv('../data/processed/goalie_final_merge.csv')
defenders_df = pd.read_csv('../data/processed/epl_defender_predictions.csv')
midfielders_df = pd.read_csv('../data/processed/epl_midfielder_predictions.csv')
forwards_df = pd.read_csv('../data/processed/epl_forward_predictions.csv')

# Function to get the full photo URL for a player
def get_photo_url(photo_id):
    return f"{photo_base_url}{photo_id}.png"

# Sort midfielders by EPL_PREDICTIONS and select the top 4
top_goalies = result_df.sort_values(by='epl_predictions', ascending=False).head(4)
top_midfielders = result_df.sort_values(by='epl_predictions', ascending=False).head(4)
top_defenders = result_df.sort_values(by='epl_predictions', ascending=False).head(4)
top_forwards = result_df.sort_values(by='epl_predictions', ascending=False).head(4)


def get_player_image_url(player_name):
    base_url = "https://www.thesportsdb.com/api/v1/json/3/searchplayers.php"
    response = requests.get(f"{base_url}?p={player_name}")
    data = response.json()
    if data['player'] and len(data['player']) > 0:
        return data['player'][0]['strThumb']
    return None

# Plot the picture of the top 4 midfielders
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for ax, (_, player) in zip(axes, top_midfielders.iterrows()):
    player_name = f"{player['first_name_midfielder']} {player['second_name_midfielder']}"
    epl_predictions = player['epl_predictions']
    
    # Get the photo URL
    photo_url = get_player_image_url(player_name)
    print(f"Photo URL for {player_name}: {photo_url}")  # Debug print statement
    
    if photo_url:
        try:
            # Fetch and display the image
            response = requests.get(photo_url)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            
            # Display the image with the player's name and EPL predictions
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(f"{player_name}\n{player['team']}\nEPL Predictions: {epl_predictions:.2f}")
            
        except Exception as e:
            print(f"Error loading image for {player_name}: {e}")
            ax.axis('off')
            ax.text(0.5, 0.5, f"{player_name}\n{player['team']}\nEPL Predictions: {epl_predictions:.2f}\n(Image not available)",
                    ha='center', va='center', wrap=True)
    else:
        ax.axis('off')
        ax.text(0.5, 0.5, f"{player_name}\n{player['team']}\nEPL Predictions: {epl_predictions:.2f}\n(Image not available)",
                ha='center', va='center', wrap=True)

plt.tight_layout()
plt.show()

# Print detailed information about top midfielders
for _, player in top_midfielders.iterrows():
    print(f"Name: {player['first_name_midfielder']} {player['second_name_midfielder']}")
    print(f"Team: {player['team']}")
    print(f"EPL Predictions: {player['epl_predictions']:.2f}")
    print("---")



In [ ]:
#plot final team on a pitch
import matplotlib.pyplot as plt
from mplsoccer import Pitch

# Create a pitch
pitch = Pitch(pitch_type='statsbomb', pitch_color='#22312b', line_color='white')
fig, ax = pitch.draw(figsize=(16, 11))

# Define positions for each player type
positions = {
    'GK': (5, 40),
    'DEF': [(20, 15), (20, 35), (20, 55), (20, 75)],
    'MID': [(50, 20), (50, 40), (50, 60), (50, 80)],
    'FWD': [(80, 30), (80, 50), (80, 70)]
}

# Plot players on the pitch
for position, coords in positions.items():
    if position == 'GK':
        player = top_goalkeepers.iloc[0]
        x, y = coords
        ax.scatter(x, y, s=700, color='yellow', zorder=2)
        ax.text(x, y, f"{player['first_name_goalkeeper']} {player['second_name_goalkeeper']}\n{player['epl_predictions']:.2f}", 
                ha='center', va='center', fontsize=8, fontweight='bold', color='black')
    else:
        for i, (x, y) in enumerate(coords):
            if position == 'DEF':
                player = top_defenders.iloc[i]
                player_name = f"{player['first_name_defender']} {player['second_name_defender']}"
            elif position == 'MID':
                player = top_midfielders.iloc[i]
                player_name = f"{player['first_name_midfielder']} {player['second_name_midfielder']}"
            else:  # FWD
                player = top_forwards.iloc[i]
                player_name = f"{player['first_name_forward']} {player['second_name_forward']}"
            
            ax.scatter(x, y, s=700, color='yellow', zorder=2)
            ax.text(x, y, f"{player_name}\n{player['epl_predictions']:.2f}", 
                    ha='center', va='center', fontsize=8, fontweight='bold', color='black')

# Set title
plt.title("Top Fantasy Premier League Team", fontsize=20, color='white')

# Show the plot
plt.tight_layout()
plt.show()
